1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

#### Задания 2-3

In [1]:
import pandas as pd
import re
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
news = pd.read_csv("C:/Users/yana/2/articles.csv")
news.head(3)

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
users = pd.read_csv("C:/Users/yana/2/users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [4]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
with open('C:/Users/yana/2/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [5]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [6]:
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)
texts = [t for t in news['title'].values]
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


In [7]:
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10
temp_file = datapath("model.lda")
lda.save(temp_file)
lda = LdaModel.load(temp_file)

In [8]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: человек который сотрудник обнаружить район тело nn
topic_1: спутник технология запуск поверхность российский система который
topic_2: год млн компания цена суд руб который
topic_3: год погибнуть человек который это миссия лечение
topic_4: год планета достигать уголь рыба величина лауреат
topic_5: фонд экономика это турист экономический который система
topic_6: китай место японский рейтинг япония китайский бомба
topic_7: научный исследование ракета это учёный год который
topic_8: это россия который год мочь человек весь
topic_9: исследователь журнал земля который рак авария ветер
topic_10: год банк статья сша nn нефть страна
topic_11: самолёт корабль солнечный сила российский территория войско
topic_12: год учёный это который работа человек жизнь
topic_13: это снижение квартира восток год который мочь
topic_14: газ крым турция турецкий газпром астероид конкурс
topic_15: луна год компания взрыв космический двигатель полёт
topic_16: nn президент путин россия владимир nnn глава
to

In [9]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [10]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.027635,0.000000,0.215384,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.010949,0.0,0.000000,0.739258
1,4896,0.0,0.000000,0.049017,0.0,0.000000,0.0,0.000000,0.187567,0.000000,...,0.175966,0.0,0.000000,0.0,0.538640,0.0,0.000000,0.0,0.000000,0.000000
2,4897,0.0,0.000000,0.388558,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.221419,0.0,0.000000,0.0,0.000000,0.000000
3,4898,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.133401,0.162524,0.179933,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.075691,0.000000
4,4899,0.0,0.366926,0.000000,0.0,0.110014,0.0,0.000000,0.100949,0.000000,...,0.000000,0.0,0.365297,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [11]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [12]:
def get_user_embedding(user_articles_list, method):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if method=='mean':
        user_vector = np.mean(user_vector, 0)
    elif method=='median':
        user_vector = np.median(user_vector, 0)
    elif method == 'max':
        user_vector = np.max(user_vector, 0)
    return user_vector

In [13]:
# создадим таблицу для записи результатов
columns = ['Methods', 'F-Score', 'Precision', 'Recall', 'Roc_auc-score']
table = pd.DataFrame(columns=columns)

In [14]:
def embedding_process(x, metrics):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metrics), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    target = pd.read_csv("users_churn.csv")
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], test_size=0.3, random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]), f'Roc_auc_score={roc_auc_score(y_test, preds):.3f}')
    return fscore[ix], precision[ix], recall[ix],roc_auc_score(y_test, preds)

In [15]:
mean = embedding_process(x, 'mean')
median = embedding_process(x, 'median')
maximum = embedding_process(x, 'max')
table.loc[len(table)] = ['Mean', mean[0], mean[1], mean[2], mean[3]]
table.loc[len(table)] = ['Median', median[0], median[1], median[2], median[3]]
table.loc[len(table)] = ['Max', maximum[0], maximum[1], maximum[2], maximum[3]]

Best Threshold=0.255634, F-Score=0.695, Precision=0.619, Recall=0.792 Roc_auc_score=0.949
Best Threshold=0.204646, F-Score=0.648, Precision=0.537, Recall=0.816 Roc_auc_score=0.938
Best Threshold=0.294648, F-Score=0.776, Precision=0.713, Recall=0.850 Roc_auc_score=0.975


#### Задание 4

In [16]:
def get_tfidf_vector(text):
    
    vectorizer = TfidfVectorizer(max_features=25)
    vectorizer.fit_transform(text)

    output_vector = []
    for i in range(25):
        if i < len(vectorizer.idf_):
            output_vector.append(vectorizer.idf_[i])
        else:
            output_vector.append(0)
    return np.array(output_vector)

In [17]:
topic_matrix = pd.DataFrame([get_tfidf_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,4.028522,4.721669,4.721669,5.127134,5.127134,4.210844,4.210844,4.433987,4.721669,...,5.127134,5.127134,5.127134,5.127134,4.433987,5.127134,5.127134,4.433987,4.433987,4.721669
1,4896,3.662588,3.662588,4.068053,4.068053,3.374906,4.068053,4.068053,4.068053,4.068053,...,4.068053,4.068053,4.068053,4.068053,4.068053,3.662588,4.068053,4.068053,4.068053,4.068053
2,4897,4.178054,4.178054,4.178054,3.772589,3.772589,4.178054,4.178054,4.178054,4.178054,...,4.178054,4.178054,4.178054,4.178054,4.178054,4.178054,4.178054,4.178054,3.484907,4.178054
3,4898,4.390024,4.390024,4.390024,4.390024,4.390024,4.390024,4.795489,4.795489,4.795489,...,4.795489,4.795489,4.795489,4.795489,4.795489,4.795489,4.795489,4.795489,4.795489,4.390024
4,4899,3.970414,3.970414,3.970414,3.970414,3.970414,3.564949,3.970414,3.970414,3.970414,...,3.970414,3.970414,3.970414,3.970414,3.970414,3.970414,3.277267,3.564949,3.970414,3.970414


In [18]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [19]:
tfidf_mean = embedding_process(x, 'mean')
tfidf_median = embedding_process(x, 'median')
tfidf_maximum = embedding_process(x, 'max')
table.loc[len(table)] = ['TFIDF_Mean', tfidf_mean[0], tfidf_mean[1], tfidf_mean[2], tfidf_mean[3]]
table.loc[len(table)] = ['TFIDF_Median', tfidf_median[0], tfidf_median[1], tfidf_median[2], tfidf_median[3]]
table.loc[len(table)] = ['TFIDF_Max', tfidf_maximum[0], tfidf_maximum[1], tfidf_maximum[2], tfidf_maximum[3]]

Best Threshold=0.650527, F-Score=0.998, Precision=0.997, Recall=1.000 Roc_auc_score=1.000
Best Threshold=0.955627, F-Score=1.000, Precision=1.000, Recall=1.000 Roc_auc_score=1.000
Best Threshold=0.212619, F-Score=0.789, Precision=0.671, Recall=0.959 Roc_auc_score=0.975


#### Задание 5

In [20]:
table

,Methods,F-Score,Precision,Recall,Roc_auc-score
0,Mean,0.694611,0.618667,0.791809,0.949332
1,Median,0.647696,0.537079,0.815700,0.937535
2,Max,0.775701,0.713467,0.849829,0.974575
3,TFIDF_Mean,0.998296,0.996599,1.000000,0.999998
4,TFIDF_Median,1.000000,1.000000,1.000000,1.000000
5,TFIDF_Max,0.789326,0.670644,0.959044,0.974873


#### Задание 6

При различных вариантах разбиения датасета лучшие результаты были у различных способов (без tfidf). Наиболее очевидным и стабильным методом кажется медиана, т.к. она менее чувствительна к выбросам. Но, к примеру, при последнем запуске она показала наихудший результат. Модель tfidf показывает более высокие результаты, т.к. выявляет наиболее информативные документы. С медианой дает идеальные результаты 1.0 по всем метрикам, что подозрительно